In [1]:
import pickle as pkl
import time
import os
import numpy as np
import sys
from music21 import instrument, note, stream, chord, duration
from RNNmodel import create_network, sample_with_temp

import matplotlib.pyplot as plt

2023-05-03 16:38:31.629708: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 16:38:33.800858: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
with open("dictionary.pkl", "rb") as input_file:
    dictionary = pkl.load(input_file)


durations_to_int = dictionary["durations_to_int"]
int_to_duration = dictionary["int_to_duration"]
notes_to_int = dictionary["notes_to_int"]
int_to_notes = dictionary["int_to_notes"]
phrases_to_int = dictionary["phrases_to_int"]
int_to_phrases = dictionary["int_to_phrases"]

n_notes = len(notes_to_int)
n_durations = len(durations_to_int)
n_phrases = len(phrases_to_int)

In [3]:
weights_folder = 'weights'
weights_file = 'weights.h5'
embed_size = 100
rnn_units = 256
use_attention = True

model, att_model = create_network(n_notes, n_durations, n_phrases, embed_size, rnn_units, use_attention)

# Load the weights to each node
weight_source = os.path.join(weights_folder,weights_file)
model.load_weights(weight_source)
model.summary()

2023-05-03 16:38:35.997218: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-03 16:38:36.198612: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-03 16:38:36.198681: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-03 16:38:36.201735: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-03 16:38:36.201804: I tensorflow/compile

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 100)    4600        ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 100)    1900        ['input_2[0][0]']                
                                                                                              

In [4]:
import pickle
piece = None
with open('../data/data/data_germany_1.pkl', 'rb') as handle:
    piece = pickle.load(handle)

In [5]:
notes_input = [ notes_to_int[note] for note in piece[0][0] ]
print(notes_input)

[10, 15, 15, 15, 15, 15, 17, 19, 19, 19, 22, 17, 44, 22, 15, 44, 10, 15, 15, 15, 15, 17, 19, 19, 19, 22, 17, 44, 22, 15, 44, 22, 27, 26, 24, 22, 20, 19, 44, 19, 19, 44, 24, 22, 20, 44, 19, 20, 44, 24, 22, 19, 44, 18, 19, 44, 22, 27, 26, 24, 22, 20, 19, 44, 18, 19, 44, 22, 22, 24, 24, 24, 26, 24, 26, 27, 27, 44]


In [6]:
durations_input = [ durations_to_int[duration] for duration in piece[0][1] ]
print(durations_input)

[7, 7, 7, 6, 2, 4, 4, 7, 7, 7, 7, 10, 7, 7, 10, 7, 7, 7, 7, 7, 4, 4, 7, 7, 7, 7, 10, 7, 7, 10, 7, 4, 4, 13, 10, 13, 10, 10, 7, 7, 10, 10, 10, 10, 10, 7, 7, 10, 10, 10, 10, 10, 7, 7, 10, 10, 10, 10, 13, 10, 13, 10, 10, 7, 7, 10, 1, 7, 7, 12, 7, 10, 10, 10, 10, 14, 10, 10]


In [7]:
phrases_input = piece[0][2]
print(phrases_input)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [8]:
max_extra_notes = len(notes_input) - 1
seq_len = 32
phrases_output = []
phrases_output.append(0) # after the first note (that is already in the initial sequence)

notes_input_sequence = [notes_input[0]] # the first note element of the sequence
durations_input_sequence = [durations_input[0]] # the first duration element of the sequence

if seq_len is not None:
    notes_input_sequence = [129] * (seq_len - len(notes_input_sequence)) + notes_input_sequence # the sequence length must be always equal seq_len, therefore we fill the vector with values 129 (the 'START' token) at the beginning 
    durations_input_sequence = [0] * (seq_len - len(durations_input_sequence)) + durations_input_sequence # the same for durations

In [9]:
print(notes_input_sequence)

[129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 129, 10]


In [10]:
print(durations_input_sequence)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7]


In [11]:
for note_index in range(max_extra_notes):

    prediction_input = [
        np.array([notes_input_sequence])
        , np.array([durations_input_sequence])
       ]

    phrases_prediction = model.predict(prediction_input, verbose=0)
    # print(notes_input_sequence)
    # print(durations_input_sequence)
    # print(phrases_prediction[0][0])
    # print(phrases_prediction[0][1])
    
    phrases_output.append(0 if phrases_prediction[0][0] >= phrases_prediction[0][1] else 1)

    notes_input_sequence.append(notes_input[note_index])
    durations_input_sequence.append(durations_input[note_index])
    
    if len(notes_input_sequence) > seq_len:
        notes_input_sequence = notes_input_sequence[1:]
        durations_input_sequence = durations_input_sequence[1:]

2023-05-03 16:38:42.404641: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-03 16:38:42.406963: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-03 16:38:42.412132: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [12]:
print(phrases_output) # unfortunately completely wrong...

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [13]:
print(phrases_input) # should be

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
